In [164]:
import pandas as pd

https://www.bigkinds.or.kr/v2/news/에서 기간별 언론사별 다양한 뉴스정보 엑셀파일로 받을 수 있다. 조회수가 높은 네이버뉴스에서 나오는 링크가 아닌 단순한 기사의 개수만 중요시하므로 네이버에 나오는 뉴스도 확인할 필요가 있다.

In [165]:
news = pd.read_excel('.\\파이썬+텍스트+분석+입문\\파이썬 텍스트 분석 입문\\data\\NewsResult_20211120-20211121.xlsx')

C:\Users\JAEHO\anaconda3\envs\py37-64\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [169]:
# list(filter(lambda x: x.str.contains('코로나'), news['본문']))
def filter(news,*args):
    words = '|'.join([arg for arg in args])
    news = news[news['본문'].str.contains(words)]
    return news

news = filter(news,'코로나')

코로나 관련 뉴스만 분석하기위함

kwang? 로 다중인자 이해하여 만들기

In [157]:
news.reset_index(drop=True,inplace=True)

index는 초기화가 필요하다

In [158]:
from konlpy.tag import Mecab
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
import matplotlib.pyplot as plt
from wordcloud import WordCloud

In [159]:
def tokenizer(text):
    text = Mecab().nouns(text)
    return [word for word in text if len(word)>1]

In [160]:
vectorizer = TfidfVectorizer(tokenizer=tokenizer, max_df=0.95, min_df=2, max_features=5000)
DW_matrix = vectorizer.fit_transform(news['본문'])

In [161]:
nmf = NMF(n_components=5, alpha=0.2, random_state=3)
DT = nmf.fit_transform(DW_matrix)
TW = nmf.components_

C:\Users\JAEHO\anaconda3\envs\py37-64\lib\site-packages\sklearn\decomposition\_nmf.py:315: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  "'nndsvda' in 1.1 (renaming of 0.26)."), FutureWarning)


In [162]:
DT.T[1].argsort()[:5]

array([ 0, 35, 34, 33, 31], dtype=int64)

In [163]:
def print_top_words(TW, word, title,n_top_words=5):
    for topic_idx, word_vec in enumerate(TW):
        message = 'Topic %d: ' %topic_idx
        message += ' '.join(['#'+word[i] for i in word_vec.argsort()[:-n_top_words-1:-1]])
        message += '\n'+'\n'.join([title[i] for i in DT.T[topic_idx].argsort()[:-n_top_words-1:-1]])
        print(message,'\n')

print_top_words(TW, vectorizer.get_feature_names(), news['제목'])

Topic 0: #모더 #접종 #부스터 #성인 #확대
美 FDA "화이자 모더나 백신 부스터샷, 모든 성인으로 확대"
FDA, 화이자 모더나 부스터샷 모든 성인에 확대
FDA, 모든 성인에 화이자 모더나 백신 부스터샷 허용
FDA, 화이자 모더나 백신 부스터샷 18세 이상 모든 성인으로 확대
FDA, 화이자 모더나 추가 접종 모든 성인에 확대 

Topic 1: #회분 #인천 #계약 #진단 #국내
내일 화이자 백신 개별 계약물량 256.5만회 분 국내 도착 [종합]
[속보] 화이자 백신, 내일(22일) 256.5만회분 국내 들어온다
내일 화이자 백신 256만5000회분 추가 도입
[속보] 내일(22일) 화이자 백신 256만 5000회 분 국내 도입
[속보] 내일 화이자 백신 256.5만회분 국내에 도입 

Topic 2: #접종 #지사 #완료 #아스트라 #추가
김영록 지사, 화이자 백신 추가접종
김영록 지사 부스터샷 접종 "고위험군 추가접종을"
고령층 돌파감염 이 때문? 아스트라 항체, 화이자의 5분의 1
백신 접종완료자 78.8%...18세 이상 완료율 90.8%
코로나19 백신 피해 유가족 정은경 청장에 항의 

Topic 3: #기자 #앵커 #특파원 #추가 #워싱턴
美 보건당국, 화이자 모더나 백신도 추가접종 성인 확대
美 보건당국, 화이자 모더나 백신도 추가접종 성인으로 확대
美, 부스터샷 대상 확대 유럽, 결국 재봉쇄
미 FDA, 화이자 모더나 추가접종 “성인 전체로 확대”
인구 대이동 앞두고 美 부스터샷 대상 '모든 성인' 확대 

Topic 4: #권고 #부스터 #성인 #승인 #코로나
CDC "미국 성인 모두 화이자 모더나 부스터샷 맞아야" 권고
미국, 모든 성인에 코로나 부스터샷 접종 권고
美 CDC 국장, '모든 성인 부스터샷 접종' 최종 승인
미국 보건당국, 화이자 모더나 부스터샷 모든 성인으로 확대 승인
美 보건당국 "18세 이상 부스터샷 맞으라" 대상 확대 



'\s'.join은 리스트내 원소를 \s를 추가해주고 합치는것이다.